# Pandas. Группировка и агрегация.

Давайте научимся группировать и аггрегировать наши данные в pandas.

## Задача 1

Посчитаем среднее значение оценок в зависимости от пола наших студентов.

In [39]:
import pandas as pd
import numpy as np

In [48]:
students_performance = pd.read_csv('StudentsPerformance.csv')
students_performance = students_performance.rename(columns={
    'math score': 'math_score',
    'reading score': 'reading_score',
    'writing score': 'writing_score'
})
students_performance.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Для этого к нашему dataframe применим метод группировки - groupby.

In [50]:
students_performance.groupby('gender')

Результатом этой операции будет новый dataframe, теперь к нему нужно применить какую-нибудь агрегацию. Смамое простое - вызвать какую-нибудь функцию, наприме mean().

In [51]:
students_performance.groupby('gender').mean()

,math_score,reading_score,writing_score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


Но в этом случае если бы захотели применить несколько функций - это было бы уже затруднительно. поэтому лучше расписать как мы хотим сделать группировку.

In [52]:
students_performance.groupby('gender').aggregate({'math_score': 'mean', 'reading_score': 'mean'})

,math_score,reading_score
gender,,
female,63.633205,72.608108
male,68.728216,65.473029


Результатом будет также dataframe, у которого в качестве инлексов будет название переменной, по которой мы проводили агрегацию. 

Этого можно избежать, задав аргумент as_index=False.

In [54]:
students_performance.groupby('gender', as_index=False).aggregate({'math_score': 'mean', 'reading_score': 'mean'})

,gender,math_score,reading_score
0,female,63.633205,72.608108
1,male,68.728216,65.473029


Можно сделать названия колонок в новом dataframe более информативными, используя метод rename.

In [56]:
students_performance.groupby('gender', as_index=False)\
.aggregate({'math_score': 'mean', 'reading_score': 'mean'})\
.rename(columns={
    'math_score': 'mean_math_score', 
    'reading_score': 'mean_reading_score'
})

,gender,mean_math_score,mean_reading_score
0,female,63.633205,72.608108
1,male,68.728216,65.473029


Кстати, если подавать значения для ключей в виде списка, то можно, во-первых, выводить несколько статистик для одной переменной, а, во-вторых, колонки будут иметь названия в соответствии со статистикой.

Рассмотрим пример

In [58]:
students_performance.groupby('gender', as_index=False).aggregate({'math_score': ['mean', 'count', 'std'],'reading_score': ['std', 'min', 'max']})

gender math_score                  reading_score         
                mean count        std           std min  max
0  female  63.633205   518  15.491453     14.378245  17  100
1    male  68.728216   482  14.356277     13.931832  23  100

Если выводить только одну статистику, но задавать ее не как 'math_score': 'mean', а
'math_score': ['mean'], то под названием math_score будет отображаться название mean.

In [59]:
students_performance.groupby('gender', as_index=False).aggregate({'math_score': ['mean']})

,gender,math_score
,,mean
0,female,63.633205
1,male,68.728216


## Задача 2:

Группировка нашего dataframe сразу по нескольким переменным.

В таком случае нужно просто обернуть переменные, которые мы будем использовать для группировки, в список:

In [61]:
students_performance.groupby(['gender', 'race/ethnicity'], as_index=False) \
    .aggregate({'math_score': 'mean', 'reading_score': 'mean'}) \
    .rename(columns = {'math_score': 'mean_math_score', 'reading_score': 'mean_reading_score'})

,gender,race/ethnicity,mean_math_score,mean_reading_score
0,female,group A,58.527778,69.000000
1,female,group B,61.403846,71.076923
2,female,group C,62.033333,71.944444
3,female,group D,65.248062,74.046512
4,female,group E,70.811594,75.840580
5,male,group A,63.735849,61.735849
6,male,group B,65.930233,62.848837
7,male,group C,67.611511,65.424460
8,male,group D,69.413534,66.135338
9,male,group E,76.746479,70.295775


### Что такое мультииндексы (индексы из нескольких уровней)?

Если в примере выше убрать аргумент as_index=False, то индекс стал бы сложным составным:

In [62]:
mean_scores = students_performance.groupby(['gender', 'race/ethnicity']) \
    .aggregate({'math_score': 'mean', 'reading_score': 'mean'}) \
    .rename(columns = {'math_score': 'mean_math_score', 'reading_score': 'mean_reading_score'})

In [63]:
mean_scores

mean_math_score  mean_reading_score
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923
       group C               62.033333           71.944444
       group D               65.248062           74.046512
       group E               70.811594           75.840580
male   group A               63.735849           61.735849
       group B               65.930233           62.848837
       group C               67.611511           65.424460
       group D               69.413534           66.135338
       group E               76.746479           70.295775

In [64]:
mean_scores.index

MultiIndex(levels=[['female', 'male'], ['group A', 'group B', 'group C', 'group D', 'group E']],
           labels=[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]],
           names=['gender', 'race/ethnicity'])

Как видно из результата запроса у нас теперь мультииндексы, состоящие из нескольких уровней. А именно у нас теперь на более глобальном уровне выступает gender, а внутри ещё есть группировка по значению переменной `race/ethnicity`.

Такие индексы значительно усложняют работу с dataframe.

### Что мы можем делать с такими индексами?

Мы можем указать значение сначала первого, а потом второго индекса для получения нужных нам данных.

In [65]:
mean_scores.loc[('female', 'group A')]

mean_math_score       58.527778
mean_reading_score    69.000000
Name: (female, group A), dtype: float64

Если нужно достать несколько значений по такому сложному индексу, то мы должны обернуть наш сложный запрос в ещё одни квадратные скобки.

In [66]:
mean_scores.loc[[('female', 'group A'), ('female', 'group B')]]

mean_math_score  mean_reading_score
gender race/ethnicity                                     
female group A               58.527778           69.000000
       group B               61.403846           71.076923

Несомненно, такая структура сложных индексов бывает полезна когда у нас очень сложная многоуровневая структура данных. однако, когда речь идёт о несложных данных, то лучше избежать таких сложных индексов.

Заметное преимущество мультииндексов есть!

Мы помним, что колонки в dataframe - это series.

In [67]:
students_performance.math_score

0      72
1      69
2      90
3      47
4      76
5      71
6      88
7      40
8      64
9      38
10     58
11     40
12     65
13     78
14     50
15     69
16     88
17     18
18     46
19     54
20     66
21     65
22     44
23     69
24     74
25     73
26     69
27     67
28     70
29     62
       ..
970    89
971    78
972    53
973    49
974    54
975    64
976    60
977    62
978    55
979    91
980     8
981    81
982    79
983    78
984    74
985    57
986    40
987    81
988    44
989    67
990    86
991    65
992    55
993    62
994    63
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

К series можно применять крутые методы, например, вывести все уникальные значения при помощи метода unique().

In [68]:
students_performance.math_score.unique()

array([ 72,  69,  90,  47,  76,  71,  88,  40,  64,  38,  58,  65,  78,
        50,  18,  46,  54,  66,  44,  74,  73,  67,  70,  62,  63,  56,
        97,  81,  75,  57,  55,  53,  59,  82,  77,  33,  52,   0,  79,
        39,  45,  60,  61,  41,  49,  30,  80,  42,  27,  43,  68,  85,
        98,  87,  51,  99,  84,  91,  83,  89,  22, 100,  96,  94,  48,
        35,  34,  86,  92,  37,  28,  24,  26,  95,  36,  29,  32,  93,
        19,  23,   8], dtype=int64)

Или посчитать число уникальных значений:

In [69]:
students_performance.math_score.nunique()

81

Давайте посчитаем уникальные значения на пересечении двух групп.

In [70]:
students_performance.groupby(['gender', 'race/ethnicity']).math_score

Результат - сгруппированная серия - одномерный массив с информацией о группировке по двум переменным

In [71]:
students_performance.groupby(['gender', 'race/ethnicity']).math_score.nunique()

gender  race/ethnicity
female  group A           29
        group B           51
        group C           59
        group D           53
        group E           44
male    group A           38
        group B           43
        group C           56
        group D           49
        group E           38
Name: math_score, dtype: int64

При этом метод groupby() можно икомбинировать не только с агрегацией наших данных, но и с другими методами.

## Задача 3

Отберём по 5 топ студентов по математике для каждого пола.

Для этого давайте саначала отсортируем данные.

По умолчанию сортировка будет от наименьшего к наибольшему значению (для обратной сортировки задаём ascending=False).

In [73]:
students_performance.sort_values(['gender', 'math_score'])

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math_score,reading_score,writing_score
59,female,group C,some high school,free/reduced,none,0,17,10
980,female,group B,high school,free/reduced,none,8,24,23
17,female,group B,some high school,free/reduced,none,18,32,28
787,female,group B,some college,standard,none,19,38,32
145,female,group C,some college,free/reduced,none,22,39,33
842,female,group B,high school,free/reduced,completed,23,44,36
338,female,group B,some high school,free/reduced,none,24,38,27
466,female,group D,associate's degree,free/reduced,none,26,31,38
363,female,group D,some high school,free/reduced,none,27,34,32
528,female,group D,bachelor's degree,free/reduced,none,29,41,47


А теперь сгруппируем dataframe по полу(gender) и вызовем метод head().

In [75]:
students_performance.sort_values(['gender', 'math_score'], ascending=False).groupby('gender').head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math_score,reading_score,writing_score
149,male,group E,associate's degree,free/reduced,completed,100,100,93
623,male,group A,some college,standard,completed,100,96,86
625,male,group D,some college,standard,completed,100,97,99
916,male,group E,bachelor's degree,standard,completed,100,100,100
306,male,group E,some college,standard,completed,99,87,81
451,female,group E,some college,standard,none,100,92,97
458,female,group E,bachelor's degree,standard,none,100,100,100
962,female,group E,associate's degree,standard,none,100,100,100
114,female,group E,bachelor's degree,standard,completed,99,100,100
263,female,group E,high school,standard,none,99,93,90


## Как изменять исходный dataframe

### Добавление новых колонок

**1-й способ**

Мы знаем, что dataframe - это аналоги словаря, который по именам хранит пандасовские объекты series.

Давайте создадим новый столбец `total_score`, который будет хранить суммарный балл по всем трём предметам.

Поскольку series можно складывать, то первый способ - метод "в лоб".

In [76]:
students_performance['total_score'] = students_performance.math_score + students_performance.reading_score + students_performance.writing_score
students_performance.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math_score,reading_score,writing_score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


В этом выражении слева мы указываем название новой колонки, а справа просто задаём новую серию, которая будет находиться в этой колонке.

Такой способ хорош, но позволяет сделать за раз только одну колонку с данными. А иногда есть необходимость создать несколько колонок, или преобразовать их как-то более сложно.

**2-й способ**

Для таких задач используется метов assign(). Он позволяет наши данные как-то изменять.

Давайте добавим столбец total_score_log и посчитаем логарифм значения в колонке total_score.

Для этого воспользуемся функцией `log` из модуля numpy.

In [77]:
students_performance = students_performance.assign(total_score_log = np.log(students_performance.total_score))
students_performance.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math_score,reading_score,writing_score,total_score,total_score_log
0,female,group B,bachelor's degree,standard,none,72,72,74,218,5.384495
1,female,group C,some college,standard,completed,69,90,88,247,5.509388
2,female,group B,master's degree,standard,none,90,95,93,278,5.627621
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,4.997212
4,male,group C,some college,standard,none,76,78,75,229,5.433722


### Удаление ненужных колонок

Для удалений колонок используем метод drop().

axis=1 - означает, что указанный в инструкции лейбл - это не индекс по строчкам, а название колонки.

In [78]:
students_performance.drop(['total_score', 'lunch'], axis=1).head()

,gender,race/ethnicity,parental level of education,test preparation course,math_score,reading_score,writing_score,total_score_log
0,female,group B,bachelor's degree,none,72,72,74,5.384495
1,female,group C,some college,completed,69,90,88,5.509388
2,female,group B,master's degree,none,90,95,93,5.627621
3,male,group A,associate's degree,none,47,57,44,4.997212
4,male,group C,some college,none,76,78,75,5.433722
